In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/eelco/dev/PySyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=011f65f60c3640c688ad45e5b5ba7a3e in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=fcc3e091010642e3abda8aa82d908759 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.01it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27.23it/s]


Uploading: numpy-data


# Make Requests

In [8]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8981db1632d34c41b057e3a96047b044.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=cb282346e1244e29aa814d7f3a6346a2 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_8017: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_8017
override True
Computing mean...
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [9]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

## Sync to high side

In [10]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

low_state

syft.service.sync.sync_state.SyncState

In [11]:
diff_state = compare_states(low_state, high_state)

diff_state

syft.service.sync.diff_state.DiffState

In [12]:
# syft absolute
# state.objs_to_sync
from syft.client.syncing import resolve

resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

╭─ Request(36fdb8dfa83142e08c2eed618d1f70de) - State: NEW ────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭─ Low side ────────────────────────────────────────────────────────────────────────────────────────────────╮  │
│  │                                                                                                           │  │
│  │  class Request:                                                                                           │  │
│  │  ‎ ‎ ‎ ‎ request_time = 2024-02-26 15:05:22                                                                   │  │
│  │  ‎ ‎ ‎ ‎ updated_at = None                                                                                    │  │
│  │  ‎ ‎ ‎ ‎ status = RequestStatus.PENDING                                                                       │  │
│  │  ‎ ‎ ‎ ‎ changes = [                                                                                          │  │
│  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                                             │  │
│  │  ‎ ‎ ‎ ‎ ]                                                                                                    │  │
│  │  ‎ ‎ ‎ ‎ requesting_user_verify_key = f59f2aec83ce8cc22813cdb17f7ac1055784e03fd666994969724ca2f9e26a5c        │  │
│  │                                                                                                           │  │
│  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
│  ╭─ High side ───────────────────────────────────────────────────────────────────────────────────────────────╮  │
│  │ None                                                                                                      │  │
│  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ UserCode(f8d778a5aa0145c199587dba03e50ebd) - State: NEW ──────────────────────────────────────────────────╮
     │                                                                                                            │
     │  ╭─ Low side ───────────────────────────────────────────────────────────────────────────────────────────╮  │
     │  │                                                                                                      │  │
     │  │  class UserCode:                                                                                     │  │
     │  │  ‎ ‎ ‎ ‎ service_func_name = compute_mean                                                                │  │
     │  │  ‎ ‎ ‎ ‎ input_owners = [                                                                                │  │
     │  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ test_l                                                                                      │  │
     │  │  ‎ ‎ ‎ ‎ ]                                                                                               │  │
     │  │  ‎ ‎ ‎ ‎ code_status = [                                                                                 │  │
     │  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ Node: test_l, Status: pending                                                               │  │
     │  │  ‎ ‎ ‎ ‎ ]                                                                                               │  │
     │  │  ‎ ‎ ‎ ‎ worker_pool_name = default-pool                                                                 │  │
     │  │                                                                                                      │  │
     │  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
     │  ╭─ High side ──────────────────────────────────────────────────────────────────────────────────────────╮  │
     │  │ None                                                                                                 │  │
     │  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
     │                                                                                                            │
     ╰────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Decision: Syncing all objects from low side


In [13]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[syft.service.request.request.Request, syft.service.code.user_code.UserCode],
  update_objs=[],
  delete_objs=[]
)


In [14]:
client_low.apply_state(resolved_state_low)

SyftSuccess: Synced 0 items

In [15]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 2 items

# Run code high and sync back result

In [16]:
data_high = client_high.datasets[0].assets[0]

In [17]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = b6237c72205a43da9996cc2d8b8b549a
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [18]:
# wait for the result
job_high.wait().get()

SyftInfo: Node Landed!
override True


26/02/24 17:05:25 FUNCTION LOG (b6237c72205a43da9996cc2d8b8b549a): Computing mean...


17.0

In [19]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [20]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)

assert isinstance(accept_res, SyftError)
accept_res

SyftError: This ActionObject is the result of Job b6237c72205a43da9996cc2d8b8b549a, please use the `Job.info` instead.

In [21]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
Approving request for domain test_h
ADDING PERMISSION [READ: 812d0f8f4d0649ce85a0de6abcce1dec as f59f2aec83ce8cc22813cdb17f7ac1055784e03fd666994969724ca2f9e26a5c] 812d0f8f4d0649ce85a0de6abcce1dec
ActionObject 4 <class 'syft.service.action.numpy.NumpyScalarObject'> 17.0
returning existing job
setting permission
None
None
New result 17.0
Job(b6237c72205a43da9996cc2d8b8b549a) Setting new result 812d0f8f4d0649ce85a0de6abcce1dec -> 812d0f8f4d0649ce85a0de6abcce1dec


SyftSuccess: Request submitted for updating result.

In [22]:
# accept_res = request.accept_by_depositing_result(job_info.result.get())

# assert isinstance(accept_res, SyftSuccess)
# accept_res

In [23]:
# Need to refresh Job because we overwrite result
job_high = client_high.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

## Sync back to low

In [24]:
client_high.jobs[0]

```python
class Job:
    id: UID = b6237c72205a43da9996cc2d8b8b549a
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Computing mean...
JOB COMPLETED
    
```

In [25]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

high_state

syft.service.sync.sync_state.SyncState

In [26]:
diff_state_2 = compare_states(low_state, high_state)

diff_state_2

syft.service.sync.diff_state.DiffState

In [27]:
resolved_state_low, resolved_state_high = resolve(diff_state_2, decision="high")

╭─ Request(36fdb8dfa83142e08c2eed618d1f70de) - State: DIFF ───────────────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭─ Low side ────────────────────────────────────────────────────────────────────────────────────────────────╮  │
│  │                                                                                                           │  │
│  │  class Request:                                                                                           │  │
│  │  ‎ ‎ ‎ ‎ updated_at=None,                                                                                     │  │
│  │  ‎ ‎ ‎ ‎ change=[                                                                                             │  │
│  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                                             │  │
│  │  ‎ ‎ ‎ ‎ ]                                                                                                    │  │
│  │                                                                                                           │  │
│  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
│  ╭─ High side ───────────────────────────────────────────────────────────────────────────────────────────────╮  │
│  │                                                                                                           │  │
│  │  class Request:                                                                                           │  │
│  │  ‎ ‎ ‎ ‎ updated_at=2024-02-26 15:05:28,                                                                      │  │
│  │  ‎ ‎ ‎ ‎ change=[                                                                                             │  │
│  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                                             │  │
│  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ syft.service.request.request.ObjectMutation                                                      │  │
│  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ syft.service.request.request.ActionStoreChange                                                   │  │
│  │  ‎ ‎ ‎ ‎ ]                                                                                                    │  │
│  │                                                                                                           │  │
│  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ UserCode(f8d778a5aa0145c199587dba03e50ebd) - State: DIFF ─────────────────────────────────────────────────╮
     │                                                                                                            │
     │  ╭─ Low side ───────────────────────────────────────────────────────────────────────────────────────────╮  │
     │  │                                                                                                      │  │
     │  │  class UserCode:                                                                                     │  │
     │  │  ‎ ‎ ‎ ‎ output_policy_state=b'',                                                                        │  │
     │  │  ‎ ‎ ‎ ‎ status=(<UserCodeStatus.PENDING: 'pending'>, '')                                                │  │
     │  │                                                                                                      │  │
     │  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
     │  ╭─ High side ──────────────────────────────────────────────────────────────────────────────────────────╮  │
     │  │                                                                                                      │  │
     │  │  class UserCode:                                                                                     │  │
     │  │  ‎ ‎ ‎ ‎ output_policy_state=b'\x00\x00\x00\x00\xb9\x01\x00\x00\x00\x00\x00\x00...,                      │  │
     │  │  ‎ ‎ ‎ ‎ status=(<UserCodeStatus.APPROVED: 'approved'>, '')                                              │  │
     │  │                                                                                                      │  │
     │  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
     │                                                                                                            │
     ╰────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Job(b6237c72205a43da9996cc2d8b8b549a) - State: NEW ──────────────────────────────────────────────────╮
          │                                                                                                       │
          │  ╭─ Low side ──────────────────────────────────────────────────────────────────────────────────────╮  │
          │  │ None                                                                                            │  │
          │  ╰─────────────────────────────────────────────────────────────────────────────────────────────────╯  │
          │  ╭─ High side ─────────────────────────────────────────────────────────────────────────────────────╮  │
          │  │                                                                                                 │  │
          │  │  class Job:                                                                                     │  │
          │  │  ‎ ‎ ‎ ‎ id = b6237c72205a43da9996cc2d8b8b549a                                                      │  │
          │  │  ‎ ‎ ‎ ‎ result = 17.0                                                                              │  │
          │  │  ‎ ‎ ‎ ‎ resolved = True                                                                            │  │
          │  │  ‎ ‎ ‎ ‎ progress =                                                                                 │  │
          │  │  ‎ ‎ ‎ ‎ creation_time = 2024-02-26 17:05:23.649844                                                 │  │
          │  │                                                                                                 │  │
          │  ╰─────────────────────────────────────────────────────────────────────────────────────────────────╯  │
          │                                                                                                       │
          ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ ActionObject(812d0f8f4d0649ce85a0de6abcce1dec) - State: NEW ────────────────────────────────────╮
               │                                                                                                  │
               │  ╭─ Low side ─────────────────────────────────────────────────────────────────────────────────╮  │
               │  │ None                                                                                       │  │
               │  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  │
               │  ╭─ High side ────────────────────────────────────────────────────────────────────────────────╮  │
               │  │                                                                                            │  │
               │  │  Pointer:                                                                                  │  │
               │  │  17.0                                                                                      │  │
               │  │                                                                                            │  │
               │  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  │
               │                                                                                                  │
               ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ SyftLog(b397bbb8cbd34d5f9357b0f0e7d82a14) - State: NEW ─────────────────────────────────────────╮
               │                                                                                                  │
               │  ╭─ Low side ─────────────────────────────────────────────────────────────────────────────────╮  │
               │  │ None                                                                                       │  │
               │  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  │
               │  ╭─ High side ────────────────────────────────────────────────────────────────────────────────╮  │
               │  │                                                                                            │  │
               │  │  class SyftLog:                                                                            │  │
               │  │  ‎ ‎ ‎ ‎ stdout = Computing mean...                                                            │  │
               │  │                                                                                            │  │
               │  │  ‎ ‎ ‎ ‎ stderr =                                                                              │  │
               │  │                                                                                            │  │
               │  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  │
               │                                                                                                  │
               ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Decision: Syncing all objects from high side


In [28]:
print(resolved_state_low)

ResolvedSyncState(
  create_objs=[syft.service.job.job_stash.Job, Pointer:
17.0, syft.service.log.log.SyftLog],
  update_objs=[syft.service.request.request.Request, syft.service.code.user_code.UserCode],
  delete_objs=[]
)


In [29]:
client_low.apply_state(resolved_state_low)

syncing obj with blob id 5bed6d8abe414526b697ca8fd5661b5a
Created 17.0 blob_id: 5bed6d8abe414526b697ca8fd5661b5a


SyftSuccess: Synced 5 items

In [30]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 0 items

In [31]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

# Run code low

## Run

In [32]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low.get()

override False


17.0

In [33]:
res_low.get()

17.0

In [34]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_policy.last_output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

NameError: name 'code' is not defined

In [35]:
private_high.mean()

17.0

In [36]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)

job_low

f8d778a5aa0145c199587dba03e50ebd


SyftWarning: There are existing jobs for this user code, returning the latest one

```python
class Job:
    id: UID = b6237c72205a43da9996cc2d8b8b549a
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Log b397bbb8cbd34d5f9357b0f0e7d82a14 not available
JOB COMPLETED
    
```

In [38]:
job_low.wait().get()

17.0

In [39]:
job_low.logs()

Log b397bbb8cbd34d5f9357b0f0e7d82a14 not available


In [40]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)

AssertionError: 

## TODO

- Hard-deleting objects (in a clean way)
    - We *should* be able to delete without messing up a node state, because we make a single choice for a whole hierarchy 
    - What happens if we delete a queued/running job?
- Soft-delete/archive objects
    - More work, new flag on every object in every stash
- 